# Analise e processamento dos dados recebidos

# CONFIGURAR LOG DO SISTEMA

In [1]:
import sys
from pathlib import Path

# Adiciona o diretório src ao path do Python
src_path = Path.cwd().parent / 'src'
if str(src_path) not in sys.path:
	sys.path.insert(0, str(src_path))

import logging
from datetime import datetime
from fiap import LoggerManager

log_path = Path.cwd().parent / 'logs'
LoggerManager(log_path=log_path, base_filename='data_processing')
# Dessa forma usando o logging padrão ele salva no arquivo
logging.info(f'{"=" * 50}')
logging.info(f'Início do processamento: {datetime.now()}')

2026-02-02 17:57:38,508 - INFO - Logger initialized: g:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\logs\2026-02-02_data_processing.log
2026-02-02 17:57:38,515 - INFO - ==================================================
2026-02-02 17:57:38,517 - INFO - Início do processamento: 2026-02-02 17:57:38.517198


# Transformar o arquivo xlxs em csv

In [2]:
from fiap.utils import excel_to_csv

logging.info('Iniciando a conversão de Excel para CSV')
xlxs_path = Path.cwd().parent / 'data/xlxs/BASE DE DADOS PEDE 2024 - DATATHON.xlsx'
output_path = Path.cwd().parent / 'data/csv'
excel_to_csv(
	excel_path=xlxs_path,  # ou arquivo .xlsx
	output_folder=output_path,
)

2026-02-02 17:57:38,528 - INFO - Iniciando a conversão de Excel para CSV
2026-02-02 17:57:39,478 - INFO - Processando arquivo: BASE DE DADOS PEDE 2024 - DATATHON.xlsx
2026-02-02 17:57:39,478 - INFO - Total de planilhas encontradas: 3

2026-02-02 17:57:39,975 - INFO - ✓ Planilha 'PEDE2022' convertida para: PEDE2022.csv
2026-02-02 17:57:39,975 - INFO -   Dimensões: 860 linhas x 42 colunas
2026-02-02 17:57:40,540 - INFO - ✓ Planilha 'PEDE2023' convertida para: PEDE2023.csv
2026-02-02 17:57:40,542 - INFO -   Dimensões: 1014 linhas x 48 colunas
2026-02-02 17:57:41,235 - INFO - ✓ Planilha 'PEDE2024' convertida para: PEDE2024.csv
2026-02-02 17:57:41,238 - INFO -   Dimensões: 1156 linhas x 50 colunas
2026-02-02 17:57:41,238 - INFO - 
✓ Conversão concluída! 3 arquivo(s) CSV criado(s) em: g:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\data\csv


# MONTAGEM DO DATAFRAME

In [3]:
import pandas as pd

# Caminho para os arquivos CSV
csv_path = Path.cwd().parent / 'data/csv'

# Buscar todos os arquivos CSV no diretório
csv_files = sorted([f.name for f in csv_path.glob('*.csv')])

logging.info(f'Arquivos CSV encontrados: {csv_files}')

# Dicionário para armazenar os dataframes
dataframes = {}

# Ler cada arquivo CSV e renomear as colunas
for file in csv_files:
    file_path = csv_path / file
    file_name = file.replace('.csv', '')  # Nome sem extensão (ex: PEDE2022)
    
    logging.info(f'Lendo arquivo: {file}')
    df_temp = pd.read_csv(file_path)
    
    # Renomear colunas (exceto RA) adicionando o nome do arquivo como prefixo
    columns_rename = {col: f'{file_name}_{col}' for col in df_temp.columns if col != 'RA'}
    df_temp.rename(columns=columns_rename, inplace=True)
    
    dataframes[file_name] = df_temp
    logging.info(f'{file_name}: {df_temp.shape[0]} linhas, {df_temp.shape[1]} colunas')

# Fazer o merge dos dataframes pela coluna RA (outer join para incluir todos os alunos)
df = list(dataframes.values())[0]  # Começa com o primeiro dataframe
for i in range(1, len(dataframes)):
    df = df.merge(list(dataframes.values())[i], on='RA', how='outer')

logging.info(f'DataFrame final: {df.shape[0]} linhas, {df.shape[1]} colunas')
df.head()


2026-02-02 17:57:41,268 - INFO - Arquivos CSV encontrados: ['PEDE2022.csv', 'PEDE2023.csv', 'PEDE2024.csv']
2026-02-02 17:57:41,271 - INFO - Lendo arquivo: PEDE2022.csv
2026-02-02 17:57:41,392 - INFO - PEDE2022: 860 linhas, 42 colunas
2026-02-02 17:57:41,392 - INFO - Lendo arquivo: PEDE2023.csv
2026-02-02 17:57:41,460 - INFO - PEDE2023: 1014 linhas, 48 colunas
2026-02-02 17:57:41,462 - INFO - Lendo arquivo: PEDE2024.csv
2026-02-02 17:57:41,562 - INFO - PEDE2024: 1156 linhas, 50 colunas
2026-02-02 17:57:41,624 - INFO - DataFrame final: 1661 linhas, 138 colunas


,RA,PEDE2022_Fase,PEDE2022_Turma,PEDE2022_Nome,PEDE2022_Ano nasc,PEDE2022_Idade 22,PEDE2022_Gênero,PEDE2022_Ano ingresso,PEDE2022_Instituição de ensino,PEDE2022_Pedra 20,...,PEDE2024_IPV,PEDE2024_IAN,PEDE2024_Fase Ideal,PEDE2024_Defasagem,PEDE2024_Destaque IEG,PEDE2024_Destaque IDA,PEDE2024_Destaque IPV,PEDE2024_Escola,PEDE2024_Ativo/ Inativo,PEDE2024_Ativo/ Inativo.1
0,RA-1,7.0,A,Aluno-1,2003.0,19.0,Menina,2016.0,Escola Pública,Ametista,...,NaN,10.0,Fase 8 (Universitários),0.0,NaN,NaN,NaN,Universidade Santo Amaro (UNISA),Cursando,Cursando
1,RA-10,7.0,A,Aluno-10,2004.0,18.0,Menina,2021.0,Escola Pública,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RA-100,4.0,A,Aluno-100,2009.0,13.0,Menina,2019.0,Rede Decisão,Ametista,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RA-1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.835,10.0,Fase 1 (3° e 4° ano),0.0,NaN,NaN,NaN,EE Helio Luiz Dobrochinski Prof,Cursando,Cursando
4,RA-1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.920,5.0,Fase 2 (5° e 6° ano),-1.0,NaN,NaN,NaN,EE Helio Luiz Dobrochinski Prof,Cursando,Cursando


# CONSOLIDAR E PADRONIZAR COLUNAS